In [4]:
from tqdm import tqdm
import requests
import pandas as pd
import numpy as np

In [9]:
assays_binders = [
    "NVS_NR_bER", #ESR1	
    "NVS_NR_hER", #ESR1	
    "NVS_NR_mERa",	#ESR1	
    "OT_ER_ERaERa_0480", # ESR1		
    "OT_ER_ERaERa_1440", # ESR1	
    "OT_ER_ERaERb_0480", # ESR1/ESR2
    "OT_ER_ERaERb_1440", # ESR1/ESR2	
    "OT_ER_ERbERb_0480", # ESR2	
    "OT_ER_ERbERb_1440", # ESR2
    "OT_ERa_GFPERaERE_0120",# ESR1	
    "OT_ERa_GFPERaERE_0480", # ESR1	
    "ATG_ERa_TRANS_up", #ESR1
    "ATG_ERE_CIS_up", #ESR1
    "ACEA_T47D_80hr_Positive", #ESR1 
]

In [19]:
assays_aa = [
    "Tox21_ERa_BLA_Agonist_ratio",	#agonist ESR1
    "Tox21_ERa_LUC_BG1_Agonist", #agonist ESR1
    "Tox21_ERa_BLA_Antagonist_ratio",	#Tox21	ESR1- antagonist ESR1
    "Tox21_ERa_LUC_BG1_Antagonist", #Tox21	ESR1	- antagonist ESR1
]

In [20]:
esr_class = [
    "agonist-ESR1",
    "agonist-ESR1",
    "antagonist-ESR1",
    "antagonist-ESR1",
]

In [21]:
base_url = "https://comptox.epa.gov/dashboard-api/ccdapp2/assay-list-details/search/by-endpointname?name=[name]&size=[size]&page=[page]"

In [ ]:
count = 0
for assay in assays_binders:
    jsons = []
    probe_resp = requests.get(base_url
                              .replace("[name]", assay.strip())
                              .replace("[page]", "1")
                              .replace("[size]", "1"))
    elements = int(probe_resp.json()['page']['totalElements'])
    pages = elements // 1000 + 1
    print(f"{assay} -> Total elements: {elements}, pages: {pages}")
    for page in range(0, pages):
        resp = requests.get(base_url
                                  .replace("[name]", assay.strip())
                                  .replace("[page]", str(page))
                                  .replace("[size]", "1000"))
        if resp.status_code == 200:
            print(f"{assay} -> HTTP{resp.status_code}, Page: {page}")
            jsons.append(resp.json())
        else:
            print(f"ERROR: {assay} -> HTTP{resp.status_code}")

    json = jsons[0]['_embedded']['assayListDetails']

    for i in range(1, len(jsons)):
        json.extend(jsons[i]['_embedded']['assayListDetails'])

    df = pd.DataFrame(json)

    assert len(df) == elements

    path = f"data/binders/{assay}-{esr_class[count]}.csv"
    #path = f"data/aa/{assay}.csv"
    print(f"Saved {assay} -> {path}")
    df.to_csv(path, index=False)
    count+=1

print(f"Number of retrieved csv-s: {count}")


Tox21_ERa_BLA_Agonist_ratio -> Total elements: 0, pages: 1
Tox21_ERa_BLA_Agonist_ratio -> HTTP200, Page: 0
Saved Tox21_ERa_BLA_Agonist_ratio -> data/aa/Tox21_ERa_BLA_Agonist_ratio.csv
Tox21_ERa_LUC_BG1_Agonist -> Total elements: 0, pages: 1
Tox21_ERa_LUC_BG1_Agonist -> HTTP200, Page: 0
Saved Tox21_ERa_LUC_BG1_Agonist -> data/aa/Tox21_ERa_LUC_BG1_Agonist.csv
Tox21_ERa_BLA_Antagonist_ratio -> Total elements: 0, pages: 1
Tox21_ERa_BLA_Antagonist_ratio -> HTTP200, Page: 0
Saved Tox21_ERa_BLA_Antagonist_ratio -> data/aa/Tox21_ERa_BLA_Antagonist_ratio.csv
Tox21_ERa_LUC_BG1_Antagonist -> Total elements: 0, pages: 1
Tox21_ERa_LUC_BG1_Antagonist -> HTTP200, Page: 0
Saved Tox21_ERa_LUC_BG1_Antagonist -> data/aa/Tox21_ERa_LUC_BG1_Antagonist.csv
Number of retrieved csv-s: 4
